<a href="https://colab.research.google.com/github/tanviredu/LLM-Engineering-Tutorial/blob/main/AIPoweredWebSummaraizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!apt-get update && apt-get install -y curl
# Download and install Ollama
!curl -fsSL https://ollama.com/install.sh | sh
# Start Ollama service in the background
!ollama serve > /dev/null 2>&1 &
# Wait for Ollama to start
import time
time.sleep(5)
# Verify installation
!curl http://localhost:11434/api/version
# Function to check if Ollama is running
def check_ollama():
    import requests
    try:
        response = requests.get('http://localhost:11434/api/version')
        if response.status_code == 200:
            print("Ollama is running successfully!")
            return True
    except:
        print("Ollama is not running. Please check the installation.")
        return False
# Check installation
check_ollama()


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,369 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,723 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/

True

In [ ]:
!pip install beautifulsoup4

In [1]:
import os
import requests
import json
from typing import List
from bs4 import BeautifulSoup

In [2]:
#optional
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [16]:
class Website:
  def __init__(self,url):
    self.url   = url
    response   = requests.get(self.url,headers=headers)
    self.body  = response.content
    soup       = BeautifulSoup(self.body,'html.parser')
    self.title = soup.title.string if soup.title else "No Title Found"
    if soup.body:
      for irr in soup.body(['script','style','img','input']):
        irr.decompose()
        self.text = soup.body.get_text(separator='\n',strip=True)
    else:
      self.text = ""
    links = []
    for link in soup.find_all('a'):
      href = link.get('href')
      if href:
        links.append(href)
    self.links = links

  def get_contents(self):
    return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"



In [22]:
web = Website("https://medium.com/@pragyashukla2580/rag-retrieval-augmented-generation-for-your-own-documents-5e024267140e")

In [23]:
web.get_contents()

'Webpage Title:\nRAG (Retrieval-Augmented Generation) for your own documents | by Pragya Shukla | Medium\nWebpage Contents:\nOpen in app\nSign up\nSign in\nWrite\nSign up\nSign in\nRAG (Retrieval-Augmented Generation) for your own documents\nPragya Shukla\n·\nFollow\n3 min read\n·\nMar 22, 2024\n--\nListen\nShare\nDefinition\nRAG (Retrieval-Augmented Generation) is an AI framework that enhances the accuracy of large language models (LLMs) by retrieving facts from external knowledge bases. It improves the quality of LLM-generated content, ensuring access to up-to-date and reliable information.\nRAG Pipeline\nRAG Flow Diagram\nRAG has 2 main components:\nIndexing:\nThis involves loading data through document loaders, breaking it into smaller sections using text splitters, calculating embeddings for these sections using language models, and ultimately storing them in a vector database.\nRetrieval and Generation:\nIn this step, the user’s query is used to retrieve relevant information from

In [24]:
system_prompt = "You are an assistant that analyzes the contents of a website and provides short summary,ignore the navigation links related and RESPOND IN MARKDOWN"


In [26]:
def user_prompt_for(website):
  user_prompt = f"You are looking at a website titled {web.title}"
  user_prompt += "The contents of this website is as follows: please provide a short summary of the website  in MARKDOWN"
  user_prompt += "if it includes news and announcement, summarize it too \n\n"
  user_prompt += web.text
  return user_prompt

In [29]:
print(user_prompt_for(web))

You are looking at a website titled RAG (Retrieval-Augmented Generation) for your own documents | by Pragya Shukla | MediumThe contents of this website is as follows: please provide a short summary of the website  in MARKDOWNif it includes news and announcement, summarize it too 

Open in app
Sign up
Sign in
Write
Sign up
Sign in
RAG (Retrieval-Augmented Generation) for your own documents
Pragya Shukla
·
Follow
3 min read
·
Mar 22, 2024
--
Listen
Share
Definition
RAG (Retrieval-Augmented Generation) is an AI framework that enhances the accuracy of large language models (LLMs) by retrieving facts from external knowledge bases. It improves the quality of LLM-generated content, ensuring access to up-to-date and reliable information.
RAG Pipeline
RAG Flow Diagram
RAG has 2 main components:
Indexing:
This involves loading data through document loaders, breaking it into smaller sections using text splitters, calculating embeddings for these sections using language models, and ultimately stor

In [28]:
system_prompt

'You are an assistant that analyzes the contents of a website and provides short summary,ignore the navigation links related and RESPOND IN MARKDOWN'

In [30]:
def messages_for(website):
  return [
    {
      "role": "system",
      "content": system_prompt
    },
    {
      "role": "user",
      "content": user_prompt_for(website)
    }
  ]

In [32]:
print(messages_for(web))

[{'role': 'system', 'content': 'You are an assistant that analyzes the contents of a website and provides short summary,ignore the navigation links related and RESPOND IN MARKDOWN'}, {'role': 'user', 'content': 'You are looking at a website titled RAG (Retrieval-Augmented Generation) for your own documents | by Pragya Shukla | MediumThe contents of this website is as follows: please provide a short summary of the website  in MARKDOWNif it includes news and announcement, summarize it too \n\nOpen in app\nSign up\nSign in\nWrite\nSign up\nSign in\nRAG (Retrieval-Augmented Generation) for your own documents\nPragya Shukla\n·\nFollow\n3 min read\n·\nMar 22, 2024\n--\nListen\nShare\nDefinition\nRAG (Retrieval-Augmented Generation) is an AI framework that enhances the accuracy of large language models (LLMs) by retrieving facts from external knowledge bases. It improves the quality of LLM-generated content, ensuring access to up-to-date and reliable information.\nRAG Pipeline\nRAG Flow Diagr

In [36]:
!ollama pull llama2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏ 538 KB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏  10 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   1% ▕▏  36 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   2% ▕▏  74 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   2% ▕▏  93 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   3% ▕▏ 124 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   4% ▕▏ 149 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   4% ▕▏ 161 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   5% ▕▏ 195 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   6% ▕▏ 227 MB/3.8 GB  212 MB/s     16spulling manife

In [38]:
from openai import OpenAI

def summarize(url):
  website = Website(url)
  MODEL = "llama2"
  openai = OpenAI(base_url="http://localhost:11434/v1",api_key="ollama")
  resposne = openai.chat.completions.create(
    model = MODEL,
    messages = messages_for(website)
  )
  return resposne.choices[0].message.content

In [39]:
summarize("https://medium.com/@pragyashukla2580/rag-retrieval-augmented-generation-for-your-own-documents-5e024267140e")

" summary:\nRAG (Retrieval-Augmented Generation) is an AI framework that enhances the accuracy of large language models (LLMs) by retrieving facts from external knowledge bases. It improves the quality of LLM-generated content, ensuring access to up-to-date and reliable information. RAG consists of two main components: indexing and retrieval and generation. The indexing step involves loading data through document loaders, breaking it into smaller sections using text splitters, calculating embeddings for these sections using language models, and storing them in a vector database. In the retrieval and generation step, the user's query is used to retrieve relevant information from documents, and an LLM is tasked with providing responses based on the retrieved contexts and the user's query. The article demonstrates how to implement RAG using the LangChain framework and shows some of its applications, such as advanced question-answering systems, content creation and summarization, conversat